## Lotto Prediction

In [2]:
import pandas as pd
import numpy as np
import keras
import plotly.graph_objects as go

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
from config import filename_excel

In [36]:
df = pd.read_excel(filename_excel)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [37]:
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')

In [38]:
filt = df['Date'] >= df['Date'].max() - pd.DateOffset(months=6)
df_year = df[filt]
df = df_year.copy()

In [39]:
df = df.sort_values(by=['Date']).reset_index(drop=True)
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Winning Numbers'], inplace=True)

In [53]:
cols = [ 'first', 'second', 'third', 'fourth', 'fifth', 'sixth' ]

for i in cols:
    num_data = df[i].values
    num_data = first_data.reshape((-1,1))
    print(num_data)

[ 3  3  7  3  6  4  7 11  6  4 18  9  7 10  7  3  5 11 16  5  5  4 18  5
 10  7  5  8  2  7  6  2  1  1  1  6  1  3  1  2  2 12 14  4  4 16  1  5
  2 11  5 13 11  5  7  2 10 11 16  5  3  5 14  3  8 15  5  5  4  3  5  8
  5 14  3 10 13  8]
[26  4 21  6 16  6 24 21 16 17 20 10  9 21 10 10  7 14 18 25  6  8 23 13
 11 10 14  9  3 15  7  4 11 12  2  9  5  4 10  6  6 18 15  5  5 20 16  8
 12 18 16 20 13 10  9  4 20 14 30  9  8  9 20 11 11 18  6  8  5  5  7 10
  7 15  5 21 17 21]
[33  5 23 26 25 17 25 22 27 26 24 13 14 23 13 25 16 15 23 26 12 29 28 24
 14 16 18 20  5 16 15 10 13 17 12 17 10 10 19 20 11 24 17 17 23 22 17  9
 14 28 17 33 14 12 11  9 31 16 31 10 20 19 22 15 15 25 10 17 16 16 16 23
  9 17  9 28 25 24]
[34 18 33 31 33 19 26 23 31 30 26 18 15 27 17 26 23 31 34 28 14 34 32 33
 15 17 25 26  9 19 35 34 14 23 30 20 30 14 25 30 22 28 19 19 31 25 20 14
 27 30 21 34 15 18 19 22 34 20 37 17 24 25 24 22 27 27 14 23 37 22 29 24
 25 20 11 32 27 27]
[38 37 34 32 34 21 27 27 33 32 30 27 19 38 2

## Data Preprocessing

In [80]:
first_data = df['sixth'].values
first_data = first_data.reshape((-1,1))

split = int(0.75*len(first_data))

first_train = first_data[:split]
first_test  = first_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

## List to Supervised Model

In [84]:
look_back = 2

train_generator = TimeseriesGenerator(first_train, first_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(first_test, first_test, length=look_back, batch_size=1)

## Recurrent Neural Network

In [87]:
def rnn():
    model = Sequential()
    model.add(
        LSTM(10,
            activation='sigmoid',
            input_shape=(look_back,1))
    )
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['acc'])

    num_epochs = 10
    model.fit_generator(train_generator, epochs=num_epochs, verbose=1)
    
    return model

## Forecasting

In [88]:
first_data = first_data.reshape((-1))
modelz = rnn()

def predict(num_prediction, model):
    prediction_list = first_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
    
    return prediction_list

def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 1
forecast = predict(num_prediction,modelz)
forecast = [ int(round(x)) for x in forecast ]
forecast_dates = predict_dates(num_prediction)

Epoch 1/10
3/3 [==============================] - 1s 349ms/step - loss: 1026.0769 - acc: 0.0000e+00
Epoch 2/10
3/3 [==============================] - 0s 4ms/step - loss: 1024.3184 - acc: 0.0000e+00
Epoch 3/10
3/3 [==============================] - 0s 5ms/step - loss: 1022.4003 - acc: 0.0000e+00
Epoch 4/10
3/3 [==============================] - 0s 5ms/step - loss: 1020.4976 - acc: 0.0000e+00
Epoch 5/10
3/3 [==============================] - 0s 3ms/step - loss: 1018.5604 - acc: 0.0000e+00
Epoch 6/10
3/3 [==============================] - 0s 3ms/step - loss: 1016.6694 - acc: 0.0000e+00
Epoch 7/10
3/3 [==============================] - 0s 4ms/step - loss: 1014.7879 - acc: 0.0000e+00
Epoch 8/10
3/3 [==============================] - 0s 4ms/step - loss: 1012.8632 - acc: 0.0000e+00
Epoch 9/10
3/3 [==============================] - 0s 6ms/step - loss: 1011.0243 - acc: 0.0000e+00
Epoch 10/10
3/3 [==============================] - 0s 6ms/step - loss: 1009.1577 - acc: 0.0000e+00


In [89]:
trace1 = go.Scatter(
    x = df['Date'],
    y = df['sixth'],
    mode = 'lines',
    name = 'Data'
)

trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Forecast'
)

layout = go.Layout(
    title = "Lotto Prediction",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "First Winning Number"}
)

fig = go.Figure(data=[trace1,trace2], layout=layout)
fig.show()

In [69]:
forecast

[21, 0]